In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
import medmnist
from medmnist import INFO
import numpy as np
from tqdm import tqdm

In [22]:
class DataSubset(Dataset):
  def __init__(self, base_dataset, inds=None, size=-1):
    self.base_dataset = base_dataset
    if inds is None:
        inds = np.random.choice(
            list(range(len(base_dataset))), size, replace=False)
    self.inds = inds

  def __getitem__(self, ind):
    self.base_ind = self.inds[ind]
    return self.base_dataset[self.base_ind]

  def __len__(self):
        return len(self.inds)

def get_data(train_labeled_inds=None, train_unlabeled_inds=None, inds_to_query=None, active_learning_iter=False):
  transform_train = transforms.Compose([
    transforms.Pad(4, padding_mode="reflect"),
    transforms.RandomCrop(28),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    lambda x: x + 3e-2 * torch.randn_like(x)
  ])

  transform_val = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((.5, .5, .5), (.5, .5, .5)),
    lambda x: x + 3e-2 * torch.randn_like(x)
  ])

  def MedMNIST(dataset, train, transforms):
    info = INFO[dataset]
    DataClass = getattr(medmnist, info['python_class'])
    return DataClass(split='train' if train else 'val', transform=transforms, download=True)

  train_class = MedMNIST(dataset='bloodmnist', train=True, transforms=transform_train) # (img_vector, label)

  train_inds = list(range(len(train_class)))
  
  # np.random.seed(0)
  # np.random.shuffle(train_inds)

  train_inds = np.array(train_inds)
  train_labels = np.array([np.squeeze(train_class[ind][1]) for ind in train_inds])

get_data()

Using downloaded and verified file: /Users/carlosgil/.medmnist/bloodmnist.npz
